In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# ----------------------------------------
# 1. CSV 데이터 불러오기
# ----------------------------------------
file_path = "electricity_data.csv"  # CSV 파일 경로
df = pd.read_csv(file_path, parse_dates=["date"])

# 결측치 제거
df = df.dropna()

# ----------------------------------------
# 2. 데이터 전처리
# ----------------------------------------
scaler = MinMaxScaler()
price_scaled = scaler.fit_transform(df["price"].values.reshape(-1, 1))

sequence_length = 10
X, y = [], []
for i in range(len(price_scaled) - sequence_length):
    X.append(price_scaled[i:i+sequence_length])
    y.append(price_scaled[i+sequence_length])
X, y = np.array(X), np.array(y)

# Train / Test split
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# ----------------------------------------
# 3. LSTM 모델 정의 & 학습
# ----------------------------------------
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=30, batch_size=16, verbose=1)

# ----------------------------------------
# 4. 예측
# ----------------------------------------
y_pred_scaled = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred_scaled)  # 스케일 되돌리기
y_test_real = scaler.inverse_transform(y_test)    # 실제 값도 되돌리기

# ----------------------------------------
# 5. 퍼센타일 구간 경계값 계산 (학습 데이터 기준)
# ----------------------------------------
boundaries = {
    "하위 25%": np.percentile(df["price"], 25),
    "하위 10%": np.percentile(df["price"], 10),
    "하위 5%" : np.percentile(df["price"], 5),
    "상위 25%": np.percentile(df["price"], 75),
    "상위 10%": np.percentile(df["price"], 90),
    "상위 5%" : np.percentile(df["price"], 95)
}

# 구간 분류 함수
def classify_price(price, bounds):
    if price >= bounds["상위 5%"]:
        return "상위 5%"
    elif price >= bounds["상위 10%"]:
        return "상위 10%"
    elif price >= bounds["상위 25%"]:
        return "상위 25%"
    elif price <= bounds["하위 5%"]:
        return "하위 5%"
    elif price <= bounds["하위 10%"]:
        return "하위 10%"
    elif price <= bounds["하위 25%"]:
        return "하위 25%"
    else:
        return "중간 구간"

# ----------------------------------------
# 6. 결과 출력
# ----------------------------------------
print("\n=== 예측 결과 샘플 ===")
for i in range(10):  # 예측 10개만 출력
    pred_price = y_pred[i][0]
    category = classify_price(pred_price, boundaries)
    print(f"예측 요금: {pred_price:,.0f}원 → {category}")
